# How to use TFRecord format
Tensorflow에서 지원하는 data format인 `TFRecord` format으로 data를 converting하고 아룰 `tf.data` 를 사용하여 load하는 방법에 대해서 정리, `TFRecord` format의 특징은 아래와 같음. mnist dataset을 `TFRecord` format으로 converting하고, 이를 `tf.data` 로 읽어들이는 예제

* Property
    + `TFRecord` format의 dataset은 하나의 memory block에 저장되므로, file이 개별로 저장되었을 경우에 비해 io가 빠르다.
    + multi-thread로 활용하기에 더 적합
* Reference
    + <https://www.tensorflow.org/api_docs/python/tf/python_io>
    + <https://www.tensorflow.org/api_guides/python/python_io>
    + <http://bcho.tistory.com/1190> 
    + <https://medium.com/trackin-datalabs/convert-to-tfrecords-dataset-2087b0ffa4f5>
    + <http://excelsior-cjh.tistory.com/161>

### Converting dataset to TFRecord format
dataset을 `TFRecord` format으로 바꾸는 과정에서 필요한 api는 아래와 같다.

* `tf.python_io.TFRecordWriter` class의 instance를 생성 
    + argument
        - path : `TFRecord` format으로 쓸 경로, 확장자는 `.tfrecords`
        - options : `tf.python_io.TFRecordOptions` class의 instance를 전달, 예제 코드에서는

    ```python
    options = tf.python_io.TFRecordOptions(compression_type = tf.python_io.TFRecordCompressionType.GZIP)
    ```

    + usage
        -  생성된 instance에서 `write` method를 이용하여, `tf.train.Example` 로 생성된 인스턴스를 path에 전달한 경로 parameter에 write한다.


* `tf.train.Example` class의 instance를 생성
    + argument
        - Features : `tf.train.Features` class의 instance를 전달
            - `tf.train.Features` 의 argument는 feature로 feature에 `tf.train.Feature` class의 instance가 `dictionary` type으로 모여진 `dictionary` 를 전달하여 instance를 생성한다. 예제 코드에서는 

    ```python
    example = tf.train.Example(features = tf.train.Features(feature = {
                'label' : tf.train.Feature(int64_list = tf.train.Int64List(value = [label])),
                'image' : tf.train.Feature(bytes_list = tf.train.BytesList(value = [image]))}))
    ```

    + usage
        - 생성된 instance에서 `SerializeToString` method를 이용하여 직렬화하고, 이를 `tf.python_io.TFRecordWriter` class에서 생성된 instance의 `write` method로 write한다.
        
### Loading dataset converted TFRecord with `tf.data`
`TFRecord` format의 dataset을 `tf.data` 를 이용하여 불러오는 과정에서 필요한 api는 아래와 같다.

* `tf.data.TFRecordDataset` class의 instance를 생성
    + argument
        - filenames : `TFRecord` format의 dataset의 경로
        - compression_type : 불러오는 `TFRecord` format dataset의 compression type을 전달
    + usage
        - 생성된 instance에서 `map` method를 활용, 이 때 `TFRecord` format을 parsing하는 함수를 작성하고 `lambda` 로 선언한 anonymous function을 이용한다. 예제 코드에서는

    ```python
    def parse_single_example(record):
        features = {'label' : tf.FixedLenFeature((), tf.int64, 0),
                    'image' : tf.FixedLenFeature((), tf.string, '')}
        parsed_features = tf.parse_single_example(serialized = record, features = features)
        image = tf.decode_raw(parsed_features.get('image'), out_type = tf.float32)
        image = tf.reshape(tensor = image, shape = [28,28,1])
        label = tf.cast(parsed_features.get('label'), dtype = tf.int32)
        return image, label
    ```

    ```python
    val = tf.data.TFRecordDataset(filenames = './mnist_validation.tfrecords', compression_type = 'GZIP')
    val = val.map(lambda record : parse_single_example(record))
    ```
    
    + 이후 과정은 `tf.data` 를 일반적으로 사용하는 방식과 같다. 아래의 링크를 참고
        - [How to simply use tf.data](https://github.com/aisolab/TIL/blob/master/Tensorflow/How%20to%20simply%20use%20tf.data.ipynb)

### Setup

In [1]:
import os, sys
import tensorflow as tf
import numpy as np

print(tf.__version__)

1.10.0


### Load and Split
`TFRecord` format으로 변환할 dataset인 mnist dataset을 load

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(dtype = np.float32).reshape(-1,28,28,1)
y_train = y_train.astype(dtype = np.int32)
X_test = X_test.astype(dtype = np.float32).reshape(-1,28,28,1)
y_test = y_test.astype(dtype = np.int32)

# training data에서 10000개의 데이터를 뽑음
val_indices = np.random.choice(range(X_train.shape[0]), size = 10000, replace = False)
X_val = X_train[val_indices] 
y_val = y_train[val_indices]
X_train = np.delete(arr = X_train, obj = val_indices, axis = 0)
y_train = np.delete(arr = y_train, obj = val_indices, axis = 0)
print(X_val.shape, y_val.shape, X_train.shape, y_val.shape)

(10000, 28, 28, 1) (10000,) (50000, 28, 28, 1) (10000,)


### Converting dataset to TFRecord format
mnist example

In [3]:
train = zip(X_train, y_train)
validation = zip(X_val, y_val)
test = zip(X_test, y_test)

split = dict(train = train, validation = validation, test = test)

In [4]:
options = tf.python_io.TFRecordOptions(compression_type = tf.python_io.TFRecordCompressionType.GZIP)

for key in split.keys():
    dataset = split.get(key)
    writer = tf.python_io.TFRecordWriter(path = './mnist_{}.tfrecords'.format(key), options = options)
    
    for data, label in dataset:
        image = data.tostring()
        example = tf.train.Example(features = tf.train.Features(feature = {
            'label' : tf.train.Feature(int64_list = tf.train.Int64List(value = [label])),
            'image' : tf.train.Feature(bytes_list = tf.train.BytesList(value = [image]))
        }))
        writer.write(example.SerializeToString())
    else:
        writer.close()
        print('{} was converted to tfrecords'.format(key))

train was converted to tfrecords
validation was converted to tfrecords
test was converted to tfrecords


### Loading dataset converted TFRecord with `tf.data`
mnist example

In [5]:
def parse_single_example(record):
    features = {'label' : tf.FixedLenFeature((), tf.int64, 0),
                'image' : tf.FixedLenFeature((), tf.string, '')}
    parsed_features = tf.parse_single_example(serialized = record, features = features)
    image = tf.decode_raw(parsed_features.get('image'), out_type = tf.float32)
    image = tf.reshape(tensor = image, shape = [28,28,1])
    label = tf.cast(parsed_features.get('label'), dtype = tf.int32)
    return image, label

In [6]:
val = tf.data.TFRecordDataset(filenames = './mnist_validation.tfrecords', compression_type = 'GZIP')
val = val.map(lambda record : parse_single_example(record))
val = val.batch(batch_size = 2)

In [7]:
val_iterator = val.make_initializable_iterator()
X, y = val_iterator.get_next()

In [8]:
with tf.Session() as sess:
    sess.run(val_iterator.initializer)
    X_data = sess.run(X)
    y_data = sess.run(y)

In [9]:
print(X_data.shape, X_data.dtype)
print(y_data.shape, y_data.dtype)

(2, 28, 28, 1) float32
(2,) int32
